In [2]:
import pandas as pd
import numpy as np
import re
import html
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report

# 1. Caricamento Dati
df = pd.read_csv('winter_project_2026/development.csv')

# 2. Preprocessing e Pulizia
def clean_text(text):
    if pd.isna(text):
        return ""
    # Decodifica entità HTML (es. &amp; -> &)
    text = html.unescape(text)
    # Rimozione tag HTML
    text = re.sub(r'<[^>]+>', ' ', text)
    # Rimozione caratteri speciali e numeri (manteniamo solo lettere)
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    # Rimozione spazi multipli e lowercase
    text = re.sub(r'\s+', ' ', text).strip().lower()
    return text

print("Inizio pulizia del testo...")
# Uniamo titolo e articolo per avere più informazione
df['text_raw'] = df['title'].fillna('') + " " + df['article'].fillna('')
df['cleaned_text'] = df['text_raw'].apply(clean_text)
print("Pulizia completata.")

# 3. Split Train/Validation
X = df['cleaned_text']
y = df['label']

# Stratify è fondamentale per il bilanciamento delle classi
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Dimensioni Train: {X_train.shape}, Dimensioni Validation: {X_val.shape}")

# 4. Vectorization (TF-IDF)
# Limitiamo a 10000 feature per velocità e memoria, rimuoviamo stop words inglesi
tfidf = TfidfVectorizer(max_features=10000, stop_words='english', ngram_range=(1, 1))

print("Vettorizzazione in corso...")
X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf = tfidf.transform(X_val)
print("Vettorizzazione completata.")

# 5. Training Modello Baseline (Logistic Regression)
# class_weight='balanced' è cruciale per la metrica Macro F1 su dataset sbilanciati
model = LogisticRegression(class_weight='balanced', solver='liblinear', random_state=42)

print("Addestramento modello...")
model.fit(X_train_tfidf, y_train)

# 6. Valutazione
y_pred = model.predict(X_val_tfidf)

macro_f1 = f1_score(y_val, y_pred, average='macro')
print(f"\n--- Risultati Baseline ---")
print(f"Macro F1 Score: {macro_f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

Inizio pulizia del testo...
Pulizia completata.
Dimensioni Train: (63997,), Dimensioni Validation: (16000,)
Vettorizzazione in corso...
Vettorizzazione completata.
Addestramento modello...

--- Risultati Baseline ---
Macro F1 Score: 0.6342

Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.66      0.67      4709
           1       0.66      0.75      0.70      2118
           2       0.69      0.74      0.71      2232
           3       0.57      0.47      0.51      1995
           4       0.74      0.92      0.82      1715
           5       0.44      0.34      0.38      2611
           6       0.53      0.80      0.64       620

    accuracy                           0.64     16000
   macro avg       0.62      0.67      0.63     16000
weighted avg       0.63      0.64      0.63     16000



In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split

# --- CONFIGURAZIONE ---
# Assicurati di scaricare i dati dal link nel PDF 
TRAIN_FILE = "development.csv"
EVAL_FILE = "evaluation.csv"

print("--- 1. Caricamento Dati ---")

# Caricamento dataset
dev_df = pd.read_csv(TRAIN_FILE)
eval_df = pd.read_csv(EVAL_FILE)

# Pre-elaborazione basilare: Unione di Titolo e Articolo
# Questo è simile a prendere l'intero segnale audio. Il titolo spesso contiene parole chiave forti.
# Riempiamo eventuali NaN con stringhe vuote per evitare errori
dev_df['text_combined'] = dev_df['title'].fillna('') + " " + dev_df['article'].fillna('')
eval_df['text_combined'] = eval_df['title'].fillna('') + " " + eval_df['article'].fillna('')

# Estrazione Label (y) e Dati (X grezzi)
y_dev = dev_df['label'].values
X_dev_raw = dev_df['text_combined'].values
X_eval_raw = eval_df['text_combined'].values
ids_eval = eval_df['id'].values # [cite: 18]

print(f"Caricati {len(dev_df)} record di DEV e {len(eval_df)} di EVAL.")
print("Esempio testo:", X_dev_raw[0][:100], "...")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords

# Scarica le stop words (parole comuni come "the", "is", "in" che non portano info)
nltk.download('stopwords')
stop_words = list(stopwords.words('english'))

def clean_text(text):
    """
    Pulisce il testo: lowercase, rimuove caratteri speciali.
    Equivalente al pre-processing del segnale audio.
    """
    text = str(text).lower()
    text = re.sub(r'[^a-z\s]', '', text) # Mantiene solo lettere
    return text

print("\n--- 2. Estrazione Feature (TF-IDF) ---")

# Applicazione pulizia
X_dev_clean = [clean_text(text) for text in X_dev_raw]
X_eval_clean = [clean_text(text) for text in X_eval_raw]

# Vettorizzazione TF-IDF
# max_features=5000 limita il vocabolario alle 5000 parole più importanti (riduce dimensionalità)
vectorizer = TfidfVectorizer(
    stop_words=stop_words, 
    max_features=5000, 
    ngram_range=(1, 1) # Considera singole parole. Metti (1,2) per bigrammi
)

# Fit solo sul train, transform su entrambi
X_dev_features = vectorizer.fit_transform(X_dev_clean)
X_eval_features = vectorizer.transform(X_eval_clean)

print(f"Feature DEV (X): {X_dev_features.shape}")
print(f"Feature EVAL (X_test): {X_eval_features.shape}")
# Nota: TF-IDF restituisce matrici sparse, ottime per la memoria

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

print("\n--- 3. Addestramento e Tuning (SVM) ---")

# Suddivisione Train/Val (stratify è importante per mantenere le proporzioni delle classi)
X_train, X_val, y_train, y_val = train_test_split(
    X_dev_features, y_dev, test_size=0.2, random_state=42, stratify=y_dev
)

model = SVC(random_state=42, class_weight='balanced') # 'balanced' aiuta se le classi sono sbilanciate

# Griglia parametri adattata per il testo
param_grid = {
    'C': [0.1, 1, 10],            
    'kernel': ['linear', 'rbf'], # Linear è spesso superiore per il testo ad alta dimensione
    # gamma è utile solo per rbf, per linear viene ignorato
}

# Metrica: Macro F1 come richiesto dal progetto [cite: 47]
scorer = make_scorer(f1_score, average='macro')

print("Inizio Grid Search (potrebbe richiedere tempo)...")
grid_search = GridSearchCV(model, param_grid, cv=3, scoring=scorer, verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

print(f"\nMigliori parametri: {grid_search.best_params_}")
print(f"Miglior CV F1 Score: {grid_search.best_score_:.4f}")

# Validazione
y_pred_val = best_model.predict(X_val)
val_f1 = f1_score(y_val, y_pred_val, average='macro')
print(f"Validation F1 Score: {val_f1:.4f}")

In [ ]:
print("\n--- 4. Generazione Submission ---")

# Predizione sul set di Evaluation (già trasformato col vectorizer al punto 2)
y_pred_eval = best_model.predict(X_eval_features)

# Creazione DataFrame
submission_df = pd.DataFrame({
    'Id': ids_eval,
    'Predicted': y_pred_eval
})

# Verifica formato
print(submission_df.head())

# Salvataggio
submission_df.to_csv('submission_news.csv', index=False)
print("File 'submission_news.csv' salvato.")